Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [2]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

67921/67921 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

,Date,Temp
0,1981-01-01,20.7
1,1981-01-02,17.9
2,1981-01-03,18.8
3,1981-01-04,14.6
4,1981-01-05,15.8


In [4]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts trainn samples:', train_data[:10])

number of samples: 3650
number of train samples: 3000
number of test samples: 650
firsts trainn samples: [20.7 17.9 18.8 14.6 15.8 15.8 15.8 17.4 21.8 20. ]


<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [5]:
# windowing function
def create_windows(data, window_size, horizon, shuffle=False):

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size + horizon - 1])

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y

In [6]:
past, future = (5, 2)
X_train, y_train = create_windows(train_data, past, future, shuffle=False )
X_test, y_test = create_windows(test_data, past, future, shuffle=False )

<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [7]:
num_features =  1

inputs = keras.layers.Input(shape=(past, num_features))

gru1 = keras.layers.GRU(32,return_sequences=True)(inputs)
gru2 = keras.layers.GRU(32)(gru1)
outputs = keras.layers.Dense(1)(gru2)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(), loss="mse")
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 5, 32)          │         3,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,729 (38.00 KB)

 Trainable params: 9,729 (38.00 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - loss: 88.5101 - val_loss: 36.4275
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 23.1973 - val_loss: 22.7882
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 17.9675 - val_loss: 19.4153
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 17.6023 - val_loss: 17.7604
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 15.2398 - val_loss: 16.4937
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 13.8894 - val_loss: 13.9893
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 12.5978 - val_loss: 12.4054
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10.6053 - val_loss: 11.1087
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10.1422 - val_loss: 10.3703
Epoch 10/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9.8001 - val_loss: 10.6112
Epoch 11/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9.5809 - val_loss: 9.4681
Epoch 12/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s

In [9]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6.7466
Test Loss: 6.9262003898620605


<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [10]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df_multi = df[['Temp', 'portion_year']].copy()

## train - test split
train_data = df_multi.iloc[:3000].copy()
test_data = df_multi.loc[3000:, :].copy()

In [11]:
## Create windows
def create_windows_multi(data, window_size, horizon, target_col_idx, shuffle=False):

    if isinstance(data, pd.DataFrame):
        data = data.values

    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size, :])
        y.append(data[i+window_size+horizon-1, target_col_idx])

    X, y = np.array(X), np.array(y)

    if shuffle:
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]

    return X, y

In [12]:
X_train, y_train = create_windows_multi(train_data, past, future, target_col_idx=0, shuffle=False )
X_test, y_test = create_windows_multi(test_data, past, future, target_col_idx=0, shuffle=False )

In [13]:
num_features = 2
inputs = keras.layers.Input(shape=(past, num_features))

gru1 = keras.layers.GRU(64, return_sequences=True)(inputs)
gru2 = keras.layers.GRU(64, return_sequences=True)(gru1)
gru3 = keras.layers.GRU(32)(gru2)

outputs = keras.layers.Dense(1)(gru3)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(), loss="mse")
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 5, 2)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 5, 64)          │        13,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 5, 64)          │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,457 (185.38 KB)

 Trainable params: 47,457 (185.38 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback], verbose=1
)

Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 85.9037 - val_loss: 36.2951
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 25.6890 - val_loss: 27.1196
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 19.7645 - val_loss: 22.2441
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 19.3266 - val_loss: 19.5170
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 17.7474 - val_loss: 18.1925
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 17.1297 - val_loss: 17.4132
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 17.1259 - val_loss: 17.1529
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 16.6397 - val_loss: 16.9753
Epoch 9/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 17.2574 - val_loss: 16.9406
Epoch 10/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 16.3960 - val_loss: 15.6905
Epoch 11/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 13.1478 - val_loss: 13.0740
Epoch 12/200
38/38 ━━━━━━━━━━━━━━━━━

In [15]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.9075
Test Loss: 6.330263614654541


<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


Las respuestas correctas son la a), b) y d)

Opción A --> Entrada: Secuencia de palabras o caracteres  Salida: Un numero de valoracion o una etiqueta de bueno o malo

Opcion B --> Entrada: Secuencia de audio  Salida: etiqueta de voz valida o no

Opcion C (Incorrecta) --> Entrada: Una nota de música Salida: La secuencia de notas para generar la música

Opción D --> Entrada: Una secuencia de piezas de música Salida: etiqueta del autor al que pertenece







<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.


Las respuestas correctas son la a), b), c) y d)

Opcion A --> One-hot encoding necesita un vector del tamaño del vocabulario mientras que word embeddings usa vectores más pequeños

Opcion B --> Los embeddings ubican a las palabras con significados similares cercanamente en el espacio vectorial mientras que one-hot encoding no relaciona similitudes

Opcion C --> Existen modelos preentrenados cuyo conocimiento puede ser reutilizado para una tarea de NLP

 Opción D --> Al estar en espacios vectoriales continuos, se pueden visualizar las relaciones mediante tecnicas de PCA